VGG16 s fine tunning

In [1]:
#!unzip -qq '/content/drive/MyDrive/Datafiles/it4n/cwe3categ.zip'
#!unzip -qq '/content/drive/MyDrive/Datafiles/it4n/cwe3categ_augmented.zip'
#!unzip -qq '/content/drive/MyDrive/Datafiles/it4n/it4n_train_reduced.zip'
#!unzip -qq '/content/drive/MyDrive/Datafiles/digits/digits_small.zip'

# TESTOVACI DATA
!wget -qq 'https://github.com/lukyfox/Datafiles/raw/master/digits/digits.zip'  
!unzip -qq '/content/digits.zip'

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, Callback
from keras import backend, Input, Model
from tensorflow.keras.applications import VGG16

from keras.metrics import Precision, Recall
from sklearn.metrics import classification_report

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter


class AccuracyCallback(Callback):
    def __init__(self, test_data, classes):
        self.test_data = test_data
        self.classes = list(classes)

    def on_epoch_end(self, epoch, logs=None):
        #batch_index = 0
        #data_list = []
        #while batch_index <= self.test_data.batch_index:
        #  data = self.test_data.next()
        #  data_list.append(data[0])
        #  batch_index += 1

        # now, data_array is the numeric data of whole images
        #x_data = np.asarray(data_list)
        #y_data = self.test_data.classes

        x_data, y_data = self.test_data
        correct = 0
        incorrect = 0

        x_result = self.model.predict(x_data, verbose=0)

        class_correct = [0] * len(self.classes)
        class_incorrect = [0] * len(self.classes)

        for i in range(len(x_data)):
            x = x_data[i]
            y = y_data[i]

            res = x_result[i]

            actual_label = np.argmax(y)
            #print('actual_label =',actual_label)
            pred_label = np.argmax(res)
            #print('pred_label =', pred_label)

            if(pred_label == actual_label):
                class_correct[actual_label] += 1   
                correct += 1
            else:
                class_incorrect[actual_label] += 1
                incorrect += 1
        print('\tclass_correct =', class_correct)
        print('\tclass_incorrect =', class_incorrect)
        print("\tCorrect: %d" %(correct))
        print("\tIncorrect: %d" %(incorrect))

        for i in range(len(self.classes)):
            tot = float(class_correct[i] + class_incorrect[i])
            print(f'tot({tot}) = float(class_correct[i]({class_correct[i]}) + class_incorrect[i]({class_incorrect[i]}))')
            class_acc = -1
            if (tot > 0):
                class_acc = float(class_correct[i]) / tot
                print(f'class_acc({class_acc}) = float(class_correct[i])({class_correct[i]}) / tot({tot})')

            print("\tself.classes[i] = %s: class_acc = %.3f" %(self.classes[i],class_acc)) 

        acc = float(correct) / float(correct + incorrect)  
        print(f'acc({acc}) = float(correct)({correct}) / float(correct + incorrect)({correct+incorrect})')

        print("\tCurrent Network Accuracy: %.3f" %(acc))

'''
def extract_features_from_conv_layers(image_flow, conv_base):
  # vytahnu si data z adresaru do kategorizovaneho toku
  #counter = Counter(imageflow.classes)
  #print(counter.items())
  #class_dict = imageflow.class_indices
  # zjistim pocet samplu v source_dir
  sample_count = imageflow.samples
  # vytvorim nulovy tenzor o rozmerech (sample_count, 10, 10, 512) - 3 posledni 
  # cisla vychazeji ze shapu vystupni konvolucni VGG16 vrstvy (info ze summary) 
  features = np.zeros(shape=(sample_count, 10, 10, 512))
  # udelam to same pro labely, ty jsou vlastne jen numpy polem
  labels = np.zeros(shape=(sample_count,len(class_dict.keys())))
  # a v cyklu plnim features a labels prepocitanymi features z konvolucni baze,
  # nove features (a nezmenene labels) jsou vstupem do vlastniho klasifikatoru  
  i = 0
  for input_batch, label_batch in imageflow:
    features[i*batch_size : (i+1)*batch_size] = conv_base.predict(input_batch)
    labels[i*batch_size : (i+1)*batch_size] = label_batch
    i += 1
    # iterace v generatoru bezi do nekonecna (jestli tomu spravne rozumim, 
    # flow_from_directory prohledava cilove adresare v nekonecne smycce), proto
    # musim beh ukoncit, jakmile zpracuju posledni sample
    #print('samples processed:', i*batch_size, 'of', sample_count, 'label_batch =', label_batch)
    if i*batch_size >= sample_count:
      break

  return features, labels
'''


# reset all states, mozna to k nicemu neni, ale pro ten pocit...
backend.clear_session()
batch_size = 20
epochs = 20
image_shape = (330, 330, 3)
#image_shape = (32, 32, 3)

#train_path = '/content/_splitted/train'
train_path = '/content/digits/train'
#train_path = '/content/cwe3categ_augmented/train'
#train_path = '/content/it4n_balanced'
#train_path = '/content/drive/MyDrive/Datafiles/cwe_dataset/train'
#train_path = '/content/drive/MyDrive/Datafiles/it4n_2categs/train'

#validation_path = '/content/_splitted/validation'
validation_path = '/content/digits/validation'
#validation_path = '/content/cwe3categ_augmented/validation'
#validation_path = '/content/drive/MyDrive/Datafiles/cwe_dataset/validation'
#validation_path = '/content/drive/MyDrive/Datafiles/it4n_2categs/validation'

conv_base = VGG16(
    weights='imagenet', 
    include_top=False, 
    #input_shape=image_shape
    input_tensor=Input(shape=image_shape))


# 1. nacteni dat a jejich prepocet dle konvoluce z VGG16
# ImageDataGenerator musim mit pro train i validation kvuli augmentaci.
train_imagedatagen = ImageDataGenerator(#rescale=1./255, 
                                  rotation_range=30,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
# pro validaci nejsou nahodne transformace potreba
validation_imagedatagen = ImageDataGenerator()
# nastavim prumerne hodnoty (z ImageNet) pro preprocessing - pricteni k pixelum obrazku,
# opet pouze pro FT, u EF by to nebylo k nicemu 
train_imagedatagen.mean = np.array([123.68, 116.779, 103.939], dtype='float32')
validation_imagedatagen.mean = np.array([123.68, 116.779, 103.939], dtype='float32')

train_imageflow = train_imagedatagen.flow_from_directory(
      train_path, 
      # color mode rgb nechavam u VGG16 i pro mnist
      color_mode= 'rgb', 
      batch_size=batch_size,
      class_mode='categorical',
      target_size = image_shape[:2])
  
train_counter = Counter(train_imageflow.classes)
print(train_counter.items())
train_class_dict = train_imageflow.class_indices
train_sample_count = train_imageflow.samples

validation_imageflow = validation_imagedatagen.flow_from_directory(
      validation_path, 
      # color mode rgb nechavam u VGG16 i pro mnist
      color_mode= 'rgb', 
      batch_size=batch_size,
      class_mode='categorical',
      target_size = image_shape[:2])
  
validation_counter = Counter(validation_imageflow.classes)
print(validation_counter.items())
validation_class_dict = validation_imageflow.class_indices
validation_sample_count = validation_imageflow.samples

head_model = conv_base.output
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(len(train_counter.keys()), activation='softmax')(head_model)

model = Model(inputs=conv_base.input, outputs=head_model)

for layer in conv_base.layers:
  layer.trainable = False
# vypis vrstev modelu
#conv_base.summary()

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

batch_index = 0
data_list = []
while batch_index <= validation_imageflow.batch_index:
  data = validation_imageflow.next()
  data_list.append(data[0])
  batch_index += 1

# now, data_array is the numeric data of whole images
#x_data = []
#y_data = []
#while True:
#  try:
#    n = next(validation_imageflow)
#    print(f'({n[0]}, {n[1]})')
#    x_data.append(n[0])
#    y_data.append(n[1])
#  except StopIteration:
#    print('last item reached')
#    break

#accuracy_callback = AccuracyCallback((x_data, y_data), train_counter.keys())
#class_weight = {0: 10, 1: 1.0, 2: 100.0}
class_weight = {0: 1.0, 1: 1.0, 2: 1.0}

print('head_model training started')
history = model.fit(
    x=train_imageflow,
    epochs=epochs,
    validation_data=validation_imageflow,
    callbacks=[early_stopping],
    class_weight=class_weight
    )

train_imageflow.reset()
validation_imageflow.reset()

for layer in conv_base.layers[-4:]:
  layer.trainable = True

for layer in conv_base.layers:
  print(f'{layer} is trainable = {layer.trainable}')

print('Model recompilation')
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
print('unfreezed model training started')
history = model.fit(
    x=train_imageflow,
    epochs=epochs,
    validation_data=validation_imageflow,
    callbacks=[early_stopping],
    class_weight=class_weight
    )

# 5. vypis vysledku a vizualizace

#print(classification_report(val_features, val_labels))
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
xepochs = range(1, len(accuracy)+1)

plt.plot(xepochs, accuracy, 'bo', label='Training accuracy')
plt.plot(xepochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Accuracy')
plt.figure()

plt.plot(xepochs, loss, 'bo', label='Training loss')
plt.plot(xepochs, val_loss, 'b', label='Validation loss')
plt.title('Loss')
plt.figure()

df = pd.DataFrame(history.history)
print(df)

#for data, label in validation_imageflow:
#  print('data batch shape:', data.shape)
#  plt.imshow(data[0])
#  plt.title(str(label[0]))
#  print('label batch shape:', label.shape)
#  break

plt.show()

58900480/58889256 [==============================] - 1s 0us/step
